## Squid Marking Scheme Viewer ##

This notebook reads the variant number of the written answer question from a quiz paper (via the ArUco tag) and shows you the corresponding marking scheme. You'll need a webcam (ideally an external one on a little tripod to point at your desk) and the correct `.json` file encoding the written answer question variants and their solutions.

#### Installation ####
No installation required, this is meant to run in Binder. Just execute the following code cell.

Copyright (c) 2022 by Florian Breuer (florian.breuer@newcastle.edu.au) with special thanks to Marcus Chijoff.

In [70]:
# Built-in imports:
import io
import json
import os
import threading
from time import sleep

# External imports:
import cv2
from ipywebrtc import CameraStream, ImageRecorder
import ipywidgets as widgets
import numpy as np
import PIL.Image
import PIL.ImageFilter

# Globals:
pool = []
num_variants = 0
quitnow = False
v = 0  # current variant number

# Get list of json files in current directory
json_files = [x for x in os.listdir() if x.endswith('.json')]  
if len(json_files) > 0:
    first_file = json_files[0]
else:
    first_file = 'None found!'
    
# Set up ArUco detector
aruco_variant = cv2.aruco.DICT_5X5_100
aruco = cv2.aruco.Dictionary_get(aruco_variant)
arucoParams = cv2.aruco.DetectorParameters_create()

# Set up camera
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })
image_recorder = ImageRecorder(stream=camera)
image_recorder.autosave = False
image_recorder.recording = True
    
# Define widgets:
MarkingScheme = widgets.Dropdown(options = json_files,
                                 description = 'Marking Scheme:',
                                 value = first_file)
LoadButton = widgets.Button(description = 'Load')
VariantDisplay = widgets.Label(value='Tag number:')
VariantSelector = widgets.Dropdown(options=[('None', 0)], 
                                   value=0, 
                                   description='Variant')
SolutionDisplay = widgets.HTMLMath(value='')
RubricDisplay = widgets.HTMLMath(value=r'''<b>Grading Scheme:</b><br> 
Mark the grading bubble on the Answer Sheet
with the grade letter appropriate to the student's solution:<br><br>
<ol style="list-style-type:upper-alpha">
    <li> The student demonstrates a good understanding and obtains the correct answer [3 marks]. </li>
    <li> The student demonstrates a good understanding of how to do the problem
                (some minor errors permitted) [2 marks]. </li>
    <li> The student demonstrates a partial understanding of how to do the problem [1 mark].</li>
    <li> The solution is missing, completely wrong, or shows insufficient steps to
                 convince you that they obtained the answer themselves [0 marks]. </li>
</ol>''')
CloseButton = widgets.Button(description='Quit')
ReadButton = widgets.Button(description='Read')

# RecorderDisplay = widgets.Image()

# Functions:
def load_marking_scheme(b):
    '''Load marking scheme from json file'''
    global pool
    global num_variants
    global v
    filename = MarkingScheme.value
    if filename != 'None found!':
        with open(filename, 'r') as f:
            pool = json.load(f)
        num_variants = len(pool["L"])
        VariantSelector.options = [('None', 0)] + list((f'Variant {i}', i) for i in range(1, num_variants + 1))
        if v >= 0 and v <= num_variants:
            VariantSelector.value = v
        else:
            VariantSelector.value = 0
    else:
        print('No json file to load')
        num_variants = 0
        VariantSelector.options = [('None', 0)]
        VariantSelector.value = 0
        v = 0

def close_camera(b):
    '''close camera and quit'''
    global quitnow
    camera.close_all()
#     image_recorder.close_all()
    quitnow = True
    
def blip(b):
    '''blip image recorder, causing it to grab a frame from camera'''
#     image_recorder.recording = False
    image_recorder.recording = True    
#     RecorderDisplay.value = image_recorder.image.value
    
def read_variant(b):
    '''Read variant number from ArUco tag'''
    global v
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[...,:3] # no alpha
    (corners, ids, rejected) = cv2.aruco.detectMarkers(im_array, aruco, parameters=arucoParams)
    if ids is not None:
        v = int(ids[0])
        if v >= 0 and v <= num_variants:
            VariantSelector.value = v
            update_solution(0)
        else:
            VariantSelector.value = 0
        VariantDisplay.value = f'Tag number: {v}'

def update_solution(b):
    '''show solution to current variant'''
    if v != 0:
        Q = pool['L'][v-1]
        SolutionDisplay.value = f'<b>Question</b> (variant {Q["variant_number"]})<br> {Q["question_text"]}' +\
                        f'<br><b>Solution.</b><br> {Q["solution_text"]}'
        
def select_variant(b):
    '''user has selected a different variant'''
    global v
    v = VariantSelector.value
    VariantDisplay.value = f'Variant: {v}'
    update_solution(0)
    
# Bind widget behaviour
CloseButton.on_click(close_camera)
ReadButton.on_click(blip)
LoadButton.on_click(load_marking_scheme)
image_recorder.image.observe(read_variant, 'value')
VariantSelector.observe(select_variant, 'value')

# Display widgets
display(widgets.HBox([MarkingScheme, LoadButton]))
display(camera)
# display(RecorderDisplay)
display(ReadButton)
display(widgets.HBox([VariantDisplay, VariantSelector]))
display(SolutionDisplay)
display(RubricDisplay)
display(CloseButton)

# Finally, set up a ticker that regularly blips the image_recorder
def ticker():
    global quitnow
    while not quitnow:
        blip(0)
        sleep(1.5)
    print('Good bye!')
thread = threading.Thread(target=ticker)
thread.start()



CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 640, 'height': 480}})

Button(description='Read', style=ButtonStyle())

HTMLMath(value='')

HTMLMath(value='<b>Grading Scheme:</b><br> \nMark the grading bubble on the Answer Sheet\nwith the grade lette…

Button(description='Quit', style=ButtonStyle())